# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/groupby.py:227: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/groupby.py:227: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-pack

In [6]:
print (parquet_files)

['../../05_src/data/prices/SRNE/SRNE_2014/part.0.parquet', '../../05_src/data/prices/SRNE/SRNE_2014/part.1.parquet', '../../05_src/data/prices/SRNE/SRNE_2013/part.0.parquet', '../../05_src/data/prices/SRNE/SRNE_2013/part.1.parquet', '../../05_src/data/prices/SRNE/SRNE_2012/part.0.parquet', '../../05_src/data/prices/SRNE/SRNE_2012/part.1.parquet', '../../05_src/data/prices/SRNE/SRNE_2015/part.0.parquet', '../../05_src/data/prices/SRNE/SRNE_2015/part.1.parquet', '../../05_src/data/prices/SRNE/SRNE_2008/part.0.parquet', '../../05_src/data/prices/SRNE/SRNE_2008/part.1.parquet', '../../05_src/data/prices/SRNE/SRNE_2009/part.0.parquet', '../../05_src/data/prices/SRNE/SRNE_2009/part.1.parquet', '../../05_src/data/prices/SRNE/SRNE_2007/part.0.parquet', '../../05_src/data/prices/SRNE/SRNE_2007/part.1.parquet', '../../05_src/data/prices/SRNE/SRNE_2019/part.0.parquet', '../../05_src/data/prices/SRNE/SRNE_2019/part.1.parquet', '../../05_src/data/prices/SRNE/SRNE_2010/part.0.parquet', '../../05_src


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.
(4 pt)

In [11]:
# Write your code below.
# For each ticker, add lags, returns, and hi-lo range
dd_feat = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),                              # Lag for Close
        Adj_Close_lag_1 = x['Adj Close'].shift(1),                      # Lag for Adj Close (note: space in column name)
        returns = (x['Close'] / x['Close'].shift(1)) - 1,               # Returns calculation
        hi_lo_range = x['High'] - x['Low']                              # Hi-Lo range
    )
)

print("Features created successfully!")
print(f"Number of partitions: {dd_feat.npartitions}")
print(f"Columns: {list(dd_feat.columns)}")

Features created successfully!
Number of partitions: 90
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source', 'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns', 'hi_lo_range']


/var/folders/rk/9lzkcr454bdf31w69h8wd50c0000gn/T/ipykernel_32487/856103782.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [13]:
# Write your code below.
# Convert the Dask dataframe to a pandas dataframe
df_feat = dd_feat.compute()

print(f"Converted to pandas DataFrame with shape: {df_feat.shape}")

# Sort by ticker and date to ensure proper order for rolling window calculations
df_feat = df_feat.sort_values(['Date'])  # ticker is now the index, so sort by Date only

# Add moving average of returns using a 10-day rolling window
# Group by ticker (which is now the index) to calculate moving average within each ticker separately
df_feat['returns_ma_10'] = df_feat.groupby(df_feat.index)['returns'].rolling(window=10, min_periods=1).mean().reset_index(drop=True)

print("Added 10-day moving average of returns")
print(f"Final DataFrame columns: {list(df_feat.columns)}")
print(f"Sample data:")
# Reset index temporarily to show ticker as a column for display
df_display = df_feat.reset_index()
print(df_display[['ticker', 'Date', 'returns', 'returns_ma_10']].head(15))

Converted to pandas DataFrame with shape: (344795, 13)
Added 10-day moving average of returns
Final DataFrame columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source', 'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns', 'hi_lo_range', 'returns_ma_10']
Sample data:
   ticker       Date   returns  returns_ma_10
0     AXP 1972-06-01       NaN            NaN
1     AXP 1972-06-02 -0.004357            NaN
2     AXP 1972-06-05 -0.019694            NaN
3     AXP 1972-06-06 -0.053571            NaN
4     AXP 1972-06-07 -0.011792            NaN
5     AXP 1972-06-08  0.023866            NaN
6     AXP 1972-06-09 -0.006993            NaN
7     AXP 1972-06-12  0.016432            NaN
8     AXP 1972-06-13  0.000000            NaN
9     AXP 1972-06-14 -0.011547            NaN
10    AXP 1972-06-15 -0.011682            NaN
11    AXP 1972-06-16 -0.002364            NaN
12    AXP 1972-06-19 -0.009479            NaN
13    AXP 1972-06-20  0.002392            NaN
14    AXP 1972-06-2

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not necessary to convert to pandas because Dask DataFrames support rolling window operations, such as moving averages.

It would not have been better to do it in Dask because our dataset was straightforward and didn't have a large number of rows.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.